### Truthiness

- Short circuiting occurs when an expression stops evaluating when it can guarantee the return value
- Ruby considers everything to be truthy with the exception of `false` and `nil`


In [8]:
puts false && 3/0
puts true || 3/0

false
true


While 5 evaluates as `true` and `nil` evaluates as `false`:

In [9]:
puts 5 == true
puts nil == false

false
false


### Precedence

```ruby
nil ? 1 / 0 : 1 + 2  # 3
nil && 1 / 0         # nil
5 || 1 / 0           # 5
```

- In the first case `1/0` is never evaluated because its the truthy operand for `?` - it only gets run when the value to the left of `?` is truthy.  In this case `nil` is falsy so the second operand is evaluated.
- In the second case `1/0` is never evaluated because of short circuiting; in a `&&` expression, if the first part is false there is no need to evaluate the second part.
- In the third case, `1/0` is never evaluated because of short circuiting; in a `||` expression, if the first part is true there is no need to evaluate the second part.

### How `p` interacts with `{}` and `do..end` blocks**

- Blocks have the lowest precedence of all operators.  But between `{}` and `do..end`, `{}` has a slightly higher precedence.
- The binding between a method name and a method's argument (`p` and `Array#map`) is slightly tighter than a method call and a `do..end` block.  `array.map` gets executed first, then the return value and the block gets passed to `p` as separate arguments.
- A `{}` has a higher priority which means that it binds more tightly to `array.map`.  So `array.map` is called with the block, then that return value gets passed to `p`.


```ruby
array = [1, 2, 3]

p(array.map) do |num|
  num + 1                           #  <Enumerator: [1, 2, 3]:map>
end                                 #  => <Enumerator: [1, 2, 3]:map>

p(array.map { |num| num + 1 })      # [2, 3, 4]
                                    # => [2, 3, 4]
```

### Variable Scope

- Blocks create a new scope for local variables.  The scope created by a block following a method invocations can be referred  to as an inner scope.
- Variables initialized in an outer scope can be accessed in an inner scope, but not vice versa.
- Peer scopes do not conflict which means that blocks cannot reference variables initialized in another block.
- Nested blocks create their own inner scopes.
- **Variable Shadowing**: when you use the same variable for your parameter as the variable in the outer scope, the local variable will be disregarded in the block's inner scope.

________

- While a block can access variables in the outer scope, a method definition cannot - its scope is self-contained.
- A method can only access variables that were initialized in its inner scope of that are defined as parameters.
- In situations where a local variable and a method definition (with no arguments) share the same name, Ruby will first search for the local variable and then the method with the given name.  If neither is found, a `NameError` is raised.

________________________________________________________________________________________________________________________________

- The rules of scope for a method invocation with a block remain in full effect even within a method definition.

```ruby
def some_method
  a = 1
  5.times do
    puts a
    b = 2 # Variables initialized within the inner scope of a block cannot be accessed by the outer scope.
  end

  puts a
  puts b
end
```

- A block is part of the method invocation.  In essence, the block acts as an argument to the method.  In the same way that a local vairable can be passed to a method at invocation (ex. `x = 5` .. `hello(x)`), when a method is called with a block, the block acts as an argument to that method.

```ruby
def greetings
  yield
  puts "Goodbye"
end

word = "Hello"

greetings do
  puts word
end
=> Hello
=> Goodbye
```

- `yield` controls the interaction of the method with the block.  Since the block has access to the local variable `word`, `Hello` is output when the method is invoked and the block is executed.
- When invoking a method with a block, we aren't just limited to executing code within the block.  We can also use the return value of the block to perform some other actions.

```ruby
a = "hello"
[1, 2, 3].map { |num| a }
=> ['hello', 'hello', 'hello']
```
- With `Array#map` we can use the return value fo the block to perform transformation on each element in its caller (an array).  The block also gives the `map` method access to the local variable `a`.
- So methods can access local variables passed in as arguments and through interaction with blocks.
________________________________________________________________________________________________________________________________

- Constants can be used within blocks and method definitions.  They have lexical scope.

### Pass by Reference v. Pass by Value

- Ruby exhibits a combo of pass by reference and pass by value.  Also referred to as `pass by value of the reference` or `call by sharing`.
- When you make copies of method arguments, and pass those copies to the method, since they are copies the original objects cannot be mutated. Objects passed to methods in this way are passed by value (language is using a pass by value object passing strategy).
- When you pass references to the methods the reference can be used to mutate the original object, provided the object is mutable.  Objects passed to methods in this way are passed by reference (language is using a pass by reference object passing strategy).
________________________________________________________________
- The variable doesn't actually contain the value.  Instead, it contains a pointer to a specific area in memory that contains the value.
- Some operations mutate the address space (pass by reference) while others make the variable point to a different address space (pass by value).
________________________________________________________________
```ruby
>> greeting = 'Hello'
=> "Hello"
```
- In ruby, `greeting` references or is bound to the String object `'Hello'`.
- The variable `greeting` stores the object ID of the string.

```ruby
greeting = 'Hello'
wazzup = greeting
greeting.upcase!  # Both variables references the same string and have the same object id
```

```ruby
greeting = 'Hello'
wazzup = greeting
greeting = 'Dude!' # The variable is bound to a different object through assignment. 
                   # The original object is merely disconnected from the variable. 
```
______________________________________________________________________
- In Ruby, numbers and boolean values are immutable.

```ruby
a = 3 
a *= 2
```
- This is an example of re-assignment.  The original object `3` is disconnected from the variable `a` and a new object `6` is bound to `a`.
______________________________________________________________________

In [17]:
def fix(value)
  value.upcase!
  value.concat('!')
  value
end
s = 'hello'
t = fix(s)

puts s.object_id
puts t.object_id

49260
49260


- We first initialize the local variable `s` as `s = 'Hello'`.
- Then we pass `s` to the method invocation of `fix` and bind the return value to `t`.
- The String representing `hello` is now bound to `value` in the method.  `String#upcase!` and `String#concat` both mutate the caller so the object ID of the String that references `value` and `s` will remain the same.
- `fix(s)` will return `'HELLO!'`

In [18]:
def fix(value)
  value = value.upcase
  value.concat('!')
end

s = 'hello'
t = fix(s)

"HELLO!"

- We first initialize the local variable `s` as `s = 'Hello'`.
- Then we pass `s` to the method invocation of `fix` and bind the return value to `t`.
- Within the method, a copy is made of the String referenced by `value` and the copy is mutated before being returning a reference of that copy back to `value`.  
- `value` now references a different object than it did before.

In [19]:
def fix(value)
  value = value.upcase!
  value.concat('!')
end

s = 'hello'
puts s.object_id
t = fix(s)
puts t.object_id

49280
49280


- This time, though we assigned a reference to `value`, we end up with both `s` and `t` referring to the same object. 
- The reason for this is that `String#upcase!` returns a reference to its caller, `value`. Since the reference returned by `value.upcase!` is the same, albeit **mutated**, String we started with, the assignment effectively rebinds value back to the object it was previously bound to; nothing is mutated by the assignment.